### Calculation of atomic energies from central finite differences Eq.(15)
Requires the logfiles with energies $E(V_I(\lambda \pm \Delta \lambda))$ to evaluate Eq.(16).
Then Eq.(15) is integrated numerically to yield atomic energies.

In [ ]:
import numpy as np
import importlib
import alchemical_derivatives as ad
importlib.reload(ad)

# implements get_logfile_paths to find logfiles at correct directory/filename
class SmallMol(ad.AtomicEnergy):
    def __init__(self, xyz_path, lam_vals, delta_lambda):
        self.lambda_values = np.round(lam_vals, 2)
        self.delta_lambda = 2*delta_lambda # internally delta_lambda is equal to 2*delta_lambda in Eq.(16)
        # initialize atoms
        atom_symbols, nuc_charges, positions, valence_charges = self.parse_xyz_for_CPMD_input(xyz_path)
        # store information about every atom
        self.atoms = dict()
        for a_id in atom_symbols['elIdx']:
            self.atoms[a_id] = {'idx':a_id}
        # get logfile paths
        self.get_logfile_paths()
        # save atomic energies in here
        self.atomic_energies = dict()
        
    def get_logfile_paths(self):
        """
        assume that files are stored under
        f'./CH3NH2_cfd_files/CH3NH2_lam_{lam_val}_{atom}_bw_run.log'
        and
        f'./CH3NH2_cfd_files/CH3NH2_lam_{lam_val}_{atom}_fw_run.log'
        """
        for atom in self.atoms.keys():
            self.atoms[atom]['log0'] = []
            self.atoms[atom]['log1'] = []

            for lam_val in self.lambda_values:
                if lam_val != 0.0: # derivative at lambda = 0 will be set to zero later
                    idx = self.atoms[atom]['idx']
                    self.atoms[atom]['log0'].append(f'./CH3NH2_cfd_files/CH3NH2_lam_{lam_val}_{atom}_bw_run.log')
                    self.atoms[atom]['log1'].append(f'./CH3NH2_cfd_files/CH3NH2_lam_{lam_val}_{atom}_fw_run.log')

In [ ]:
xyz_path = './CH3NH2_cfd_files/GEOMETRY.xyz'
lambda_values = np.linspace(0,1,21)
delta_lambda = 5e-5
CH3NH2 = SmallMol(xyz_path, lambda_values, delta_lambda)
# numerical integration along lambda
# other options are 'simpson' and 'trapz'
method = 'cubic_splines' 
# assumes that the derivative at lambda = 0 is zero
add_lam0 = True
# calculates the atomic electronic energies
CH3NH2.calculate_atomic_energies('electronic', method, add_lam0)
# calculates the nuclear repulsion
# due to noise in the derivatives the self-interaction and the contribution due to overlapping nuclear charge
# densities (the ESR term in the logfiles) are calculated using only the derivative at lambda=1 assuming a linear
# relation between these derivatives and lambda
CH3NH2.calculate_nuclear_repulsion_noisy(method, add_lam0)
CH3NH2.atomic_energies['total'] = CH3NH2.atomic_energies['electronic']+CH3NH2.atomic_energies['nuclear']

### Calculation of atomic energies perturbatively Eq.(15)
Requires the logfiles with energies $E(V_I(\lambda \pm \Delta \lambda),\rho_\lambda)$ to evaluate Eq.(18).
Then Eq.(15) is integrated numerically to yield atomic energies.

In [ ]:
import numpy as np
import importlib
import alchemical_derivatives as ad
importlib.reload(ad)

# implements get_logfile_paths to find logfiles at correct directory/filename
class SmallMolPert(ad.AtomicEnergyPerturbation):
    def __init__(self, xyz_path, lam_vals, delta_lambda):
        self.lambda_values = np.round(lam_vals, 2)
        self.delta_lambda = 2*delta_lambda # internally delta_lambda is equal to 2*delta_lambda in Eq.(16)
        # initialize atoms
        atom_symbols, nuc_charges, positions, valence_charges = self.parse_xyz_for_CPMD_input(xyz_path)
        # store information about every atom
        self.atoms = dict()
        for a_id in atom_symbols['elIdx']:
            self.atoms[a_id] = {'idx':a_id}
        # get logfile paths
        self.get_logfile_paths()
        # save atomic energies in here
        self.atomic_energies = dict()
        
    def get_logfile_paths(self):
        """
        assume that files are stored under
        f'./CH3NH2_pert_files/CH3NH2_lam_{lam_val}_{atom}_bw_run_pert.log'
        and
        f'./CH3NH2_pert_files/CH3NH2_lam_{lam_val}_{atom}_fw_run_pert.log'
        """
        for atom in self.atoms.keys():
            self.atoms[atom]['log0'] = []
            self.atoms[atom]['log1'] = []

            for lam_val in self.lambda_values:
                if lam_val != 0.0: # derivative at lambda = 0 will be set to zero later
                    idx = self.atoms[atom]['idx']
                    self.atoms[atom]['log0'].append(f'./CH3NH2_pert_files/CH3NH2_lam_{lam_val}_{atom}_bw_run_pert.log')
                    self.atoms[atom]['log1'].append(f'./CH3NH2_pert_files/CH3NH2_lam_{lam_val}_{atom}_fw_run_pert.log')

In [ ]:
xyz_path = './CH3NH2_pert_files/GEOMETRY.xyz'
lambda_values = np.linspace(0,1,21)
delta_lambda = 5e-5
CH3NH2_pert = SmallMolPert(xyz_path, lambda_values, delta_lambda)
# numerical integration along lambda
# other options are 'simpson' and 'trapz'
method = 'cubic_splines' 
# assumes that the derivative at lambda = 0 is zero
add_lam0 = True
# calculates the atomic electronic energies
CH3NH2_pert.calculate_atomic_energies('electronic', method, add_lam0)
# calculates the nuclear repulsion
# due to noise in the derivatives the self-interaction and the contribution due to overlapping nuclear charge
# densities (the ESR term in the logfiles) are calculated using only the derivative at lambda=1 assuming a linear
# relation between these derivatives and lambda
CH3NH2_pert.calculate_nuclear_repulsion_noisy(method, add_lam0)
CH3NH2_pert.atomic_energies['total'] = CH3NH2_pert.atomic_energies['electronic']+CH3NH2_pert.atomic_energies['nuclear']

### Calculation of atomic energies perturbatively Eq.(15) for QM9 molecules

In [ ]:
import numpy as np
import importlib
import alchemical_derivatives as ad
importlib.reload(ad)

# implements get_logfile_paths to find logfiles at correct directory/filename
class QM9Pert(ad.AtomicEnergyPerturbation):
    def __init__(self, xyz_path, lam_vals, delta_lambda, lvalstrs, compound_name):
        self.lambda_values = lam_vals
        self.lambda_strings = lvalstrs
        self.compound_name = compound_name
        self.delta_lambda = 2*delta_lambda # internally delta_lambda is equal to 2*delta_lambda in Eq.(16)
        # initialize atoms
        atom_symbols, nuc_charges, positions, valence_charges = self.parse_xyz_for_CPMD_input(xyz_path)
        # store information about every atom
        self.atoms = dict()
        for a_id in atom_symbols['elIdx']:
            self.atoms[a_id] = {'idx':a_id}
        # get logfile paths
        self.get_logfile_paths()
        # save atomic energies in here
        self.atomic_energies = dict()
        
    def get_logfile_paths(self):
        """
        assume that files are stored under
        f'./d000227_pert_files/pert_{compound_name}_{lvalstr}_{atom}_bw.log'
        and
        f'./d000227_pert_files/pert_{compound_name}_{lvalstr}_{atom}_fw.log'
        """
        for atom in self.atoms.keys():
            self.atoms[atom]['log0'] = []
            self.atoms[atom]['log1'] = []

            for lam_val, lvalstr in zip(self.lambda_values, self.lambda_strings):
                if lam_val != 0.0: # derivative at lambda = 0 will be set to zero later
                    idx = self.atoms[atom]['idx']
                    self.atoms[atom]['log0'].append(f'./d000227_pert_files/pert_{self.compound_name}_{lvalstr}_{atom}_bw.log')
                    self.atoms[atom]['log1'].append(f'./d000227_pert_files/pert_{self.compound_name}_{lvalstr}_{atom}_fw.log')

In [ ]:
xyz_path = './d000227_pert_files/dsgdb9nsd_000227.xyz'
lambda_values = np.array([0,8,15,23,30,38])/38
delta_lambda = 5e-5
lvalstrs = ['ve_0','ve_8', 've_15', 've_23', 've_30', 've_38']
compound_name = 'dsgdb9nsd_000227'
d000227 = QM9Pert(xyz_path, lambda_values, delta_lambda, lvalstrs, compound_name)
# numerical integration along lambda
# other options are 'cubic_splines' and 'trapz'
method = 'simpson' 
# assumes that the derivative at lambda = 0 is zero
add_lam0 = True
# calculates the atomic energies
d000227.calculate_atomic_energies('total', method, add_lam0)
d000227.calculate_atomic_energies('electronic', method, add_lam0)
d000227.calculate_atomic_energies('nuclear', method, add_lam0)